<a href="https://colab.research.google.com/github/rahul4070/fake-news-detection-project/blob/main/fake_news_ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# getting the test files
!wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-US,en;q=0.9" --header="Referer: https://www.kaggle.com/" "https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/8317/109592/compressed/test.csv.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1645938675&Signature=ZZP8hePb494KAbLI8jv%2FKWVAWf6WG5kfiNz%2F7%2FiILIPUlC6bDQsMuQY%2FiOcKg%2FuV9FDMy%2BhqaNtz4iteegGYA%2FKYBC0Un7sHBHgqUENkQkJ%2Bnvgd4AGTmmgZVjAUkx0kHzi0ZPQW7BE7IxnJ3XlSc0%2FAKQheDfMfWSSTdIqRTjH1VkL1HIOqabwDmRk8FgJ%2BDb3Qb4gv%2FjgZLqpGmTbOo2RBuVSG%2FNQUUWjMXXmS0pH%2FnwQbf2fQh5AkOtNnEIRlbyO%2FC3jM18fgCBDWXpAZX2bnAhBTuUUxQpfhYa4jbkckRdbjHJtxrAMLvVBiaDFjZqelZcktuliZ4sXhd2OqeA%3D%3D&response-content-disposition=attachment%3B+filename%3Dtest.csv.zip" -c -O 'test.csv.zip'

--2022-02-24 10:25:58--  https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/8317/109592/compressed/test.csv.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1645938675&Signature=ZZP8hePb494KAbLI8jv%2FKWVAWf6WG5kfiNz%2F7%2FiILIPUlC6bDQsMuQY%2FiOcKg%2FuV9FDMy%2BhqaNtz4iteegGYA%2FKYBC0Un7sHBHgqUENkQkJ%2Bnvgd4AGTmmgZVjAUkx0kHzi0ZPQW7BE7IxnJ3XlSc0%2FAKQheDfMfWSSTdIqRTjH1VkL1HIOqabwDmRk8FgJ%2BDb3Qb4gv%2FjgZLqpGmTbOo2RBuVSG%2FNQUUWjMXXmS0pH%2FnwQbf2fQh5AkOtNnEIRlbyO%2FC3jM18fgCBDWXpAZX2bnAhBTuUUxQpfhYa4jbkckRdbjHJtxrAMLvVBiaDFjZqelZcktuliZ4sXhd2OqeA%3D%3D&response-content-disposition=attachment%3B+filename%3Dtest.csv.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.203.128, 74.125.204.128, 64.233.189.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.203.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9872852 (9.4M) [application/zip]
Saving to: ‘test.csv.zip’

test.csv.zip  

In [2]:
!unzip test.csv.zip

Archive:  test.csv.zip
  inflating: test.csv                


In [10]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from scipy import sparse
import joblib
tqdm.pandas()

In [4]:
df = pd.read_csv('test.csv')
df.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


## Preprocessing, Text_cleaning and Feature engineering 

In [5]:
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [6]:
# filling the nan text with first 20 words of title feature
df.loc[df['text'].isna(), 'text'] = df.loc[df['text'].isna(), 'title']

# filling the nan title with first 20 words of text feature
df.loc[df['title'].isna(), 'title'] = df.loc[df['title'].isna(), 'text'].apply(lambda x: ' '.join(x.split()[:20]))

df.author.fillna('unk_author',inplace=True)

ps = WordNetLemmatizer()
def preprocess(text):
  x = re.sub('[^a-zA-Z0-9]', ' ', text)
  x = x.lower()
  x = x.split()
  all_stopwords = stopwords.words('english')
  all_stopwords.remove('not')
  x = [ps.lemmatize(word) for word in x if word not in all_stopwords]
  x = ' '.join(x)
  return x


df.text.fillna('', inplace=True)

df['text'] = df.text.progress_apply(lambda x: preprocess(x))
print('preprocessing of text done!')
print('-' * 100)
print()
df['title'] = df.title.progress_apply(lambda x: preprocess(x))
print('preprocessing of title done!')
print('-' * 100)
print()
df['author'] = df.author.progress_apply(lambda x: preprocess(x))
print('preprocessingof author done!')

print('Feature engineering')
# Calculating the word count of title feature
df['title_len'] = df.title.progress_apply(lambda x: len(x.split()))

# calculating the word count of text feature
df['text_len'] = df.text.progress_apply(lambda x:len(x.split()))

df['concat_title_text'] = df[['title','text']].progress_apply(lambda x: str(x[0]) + ' ' + str(x[1]),axis=1)

df['concat_title_author'] = df[['title', 'author']].progress_apply(lambda x: x[0] + ' ' + x[1],axis=1)

print('Feature engineering done!')

100%|██████████| 5200/5200 [00:25<00:00, 202.94it/s]


preprocessing of text done!
----------------------------------------------------------------------------------------------------



100%|██████████| 5200/5200 [00:01<00:00, 4332.02it/s]


preprocessing of title done!
----------------------------------------------------------------------------------------------------



100%|██████████| 5200/5200 [00:00<00:00, 6323.43it/s]


preprocessingof author done!
Feature engineering


100%|██████████| 5200/5200 [00:00<00:00, 74505.80it/s]

Feature engineering done!


## Machine leanring:- Vectorization

In [11]:
one_hot_title_text_vect = joblib.load('/content/drive/MyDrive/fake_news/fake_news/one_hot_encoder_title_text')
x_test_text_one_hot = one_hot_title_text_vect.transform(df.concat_title_text.values)

one_hot_author_vect = joblib.load('/content/drive/MyDrive/fake_news/fake_news/one_hot_encoder_title_author')
x_test_author_one_hot = one_hot_author_vect.transform(df.concat_title_author.values)


tfidf_title_text_vect = joblib.load('/content/drive/MyDrive/fake_news/fake_news/tfidf_encoder_title_text')
x_test_text_tfidf = tfidf_title_text_vect.transform(df.concat_title_text.values)

tfidf_author_vect = joblib.load('/content/drive/MyDrive/fake_news/fake_news/tfidf_encoder_title_author')
x_test_author_tfidf = tfidf_author_vect.transform(df.concat_title_author.values)

In [14]:
text_len_min_max_scaler = joblib.load('/content/drive/MyDrive/fake_news/fake_news/text_len_min_max_scaler')
text_len_values = text_len_min_max_scaler.transform(df.text_len.values.reshape(-1,1))

title_len_min_max_scaler = joblib.load('/content/drive/MyDrive/fake_news/fake_news/title_len_min_max_scaler')
title_len_values = title_len_min_max_scaler.transform(df.title_len.values.reshape(-1,1))

In [15]:
x_test_hstack_one_hot = sparse.hstack((x_test_text_one_hot, x_test_author_one_hot, \
                                text_len_values, title_len_values)).tocsr()

x_test_hstack_tfidf = sparse.hstack((x_test_text_tfidf, x_test_author_tfidf, \
                                text_len_values, title_len_values)).tocsr()

In [17]:
lr_one_hot_model = joblib.load('/content/drive/MyDrive/fake_news/fake_news/model_lr_one_hot')
lr_test_one_hot = lr_one_hot_model.predict(x_test_hstack_one_hot)

lr_tfidf_model = joblib.load('/content/drive/MyDrive/fake_news/fake_news/model_lr_tfidf')
lr_test_tfidf = lr_tfidf_model.predict(x_test_hstack_tfidf)

svm_one_hot_model = joblib.load('/content/drive/MyDrive/fake_news/fake_news/model_svc_one_hot')
svm_test_one_hot = svm_one_hot_model.predict(x_test_hstack_one_hot)

svm_tfidf_model = joblib.load('/content/drive/MyDrive/fake_news/fake_news/model_svc_tfidf')
svm_test_tfidf = svm_tfidf_model.predict(x_test_hstack_tfidf)

gbdt_one_hot_model = joblib.load('/content/drive/MyDrive/fake_news/fake_news/model_gbc_one_hot')
gbdt_test_one_hot = gbdt_one_hot_model.predict(x_test_hstack_one_hot)

gbdt_tfidf_model = joblib.load('/content/drive/MyDrive/fake_news/fake_news/model_gbc_tfifdf')
gbdt_test_tfidf = gbdt_tfidf_model.predict(x_test_hstack_tfidf)

In [24]:
dict_ = {}

In [25]:
dict_['logistic regression one_hot encoding'] = lr_test_one_hot
dict_['logistic regression tfidf encoding'] = lr_test_tfidf
dict_['SVM one_hot encoding'] = svm_test_one_hot
dict_['SVM tfidf encoding'] = svm_test_tfidf
dict_['GBDT one_hot encoding'] = gbdt_test_one_hot
dict_['GBDT tfidf encoding'] = gbdt_test_tfidf

## Vectorization:- Deep learning

In [26]:
import pandas as pd
import numpy as np
import joblib
import datetime
from tqdm import tqdm
tqdm.pandas()
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [27]:
import tensorflow 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,Dropout, Conv1D, MaxPooling1D, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam

In [28]:
df = pd.read_csv('test.csv')
df.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [29]:
df.title.fillna(' ', inplace=True)
df.text.fillna(' ', inplace=True)
df.author.fillna(' ', inplace=True)

In [30]:
df['title_author'] = df[['title', 'author']].progress_apply(lambda x: x[0] + ' ' + x[1], axis=1)
df['title_author_text'] = df[['title', 'author', 'text']].progress_apply(lambda x: x[0] + ' ' + x[1] + ' ' + ' '.join(x[2].split()[:100]), axis=1)

100%|██████████| 5200/5200 [00:00<00:00, 13506.62it/s]


In [31]:
x_test = df[['title_author', 'title_author_text']]

In [32]:
def text_to_sequences(train, test, file_name, training):
  '''
      function to perform text tokenizer 
      returns train_tokenizer and test_tokenizer
  '''

  if training:
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(train.astype(str))
    train_text = tokenizer.texts_to_sequences(train.astype(str))
    test_text  = tokenizer.texts_to_sequences(test.astype(str))
    joblib.dump(tokenizer, open('/content/drive/MyDrive/fake_news/Deep_learning/tokenizer/' + file_name , 'wb'))

    return train_text, test_text, tokenizer
  else:
    tokenizer = joblib.load(open('/content/drive/MyDrive/fake_news/Deep_learning/tokenizer/' + file_name , 'rb'))
    test_text = tokenizer.texts_to_sequences(test)
    return test_text


def text_padding(train, test, length, training):
  '''
      function to perform post-padding of sequences
      returns padding_train, padding_test
  '''
  
  if training:
    train = pad_sequences(train, maxlen=length, padding='pre')
    test  = pad_sequences(test,  maxlen=length, padding='pre')

    return train, test
  
  else:
    test_ = pad_sequences(test,  maxlen=length, padding='pre')
    
    return test_

In [33]:
x_test_author_token = text_to_sequences(x_test.title_author.values, x_test.title_author.values, 'title_author', False)
x_test_author_text_token = text_to_sequences(x_test.title_author_text.values, x_test.title_author_text.values, 'title_author_text', False)

In [34]:
max_length_author = 20
max_length_author_text = 125

# padding the text data
x_test_author_padded = text_padding(x_test_author_token, x_test_author_token, max_length_author, False)
x_test_author_text_padded = text_padding(x_test_author_token, x_test_author_token, max_length_author_text, False)

In [35]:
x_test = [x_test_author_padded, x_test_author_text_padded]

In [36]:
model_lstm = keras.models.load_model('/content/drive/MyDrive/fake_news/Deep_learning/models/model_lstm.hdf5')
lstm_predicted = model_lstm.predict(x_test)

In [43]:
lstm_predicted = ((lstm_predicted > 0.5) + 0).ravel()

In [44]:
dict_['lstm'] = lstm_predicted

In [45]:
model_cnn = keras.models.load_model('/content/drive/MyDrive/fake_news/Deep_learning/models/model_cnn.hdf5')
cnn_predicted = model_cnn.predict(x_test)

In [46]:
cnn_predicted = ((cnn_predicted > 0.5) + 0).ravel()

In [47]:
dict_['cnn'] = cnn_predicted

In [48]:
model_lstm_cnn = keras.models.load_model('/content/drive/MyDrive/fake_news/Deep_learning/models/model_lstm_cnn.hdf5')
lstm_cnn_predicted = model_lstm_cnn.predict(x_test)

In [49]:
lstm_cnn_predicted = ((lstm_cnn_predicted > 0.5) + 0).ravel()

In [50]:
dict_['lstm_cnn'] = lstm_cnn_predicted

In [52]:
predicted_df = pd.DataFrame(dict_)

In [53]:
predicted_df.head()

,logistic regression one_hot encoding,logistic regression tfidf encoding,SVM one_hot encoding,SVM tfidf encoding,GBDT one_hot encoding,GBDT tfidf encoding,lstm,cnn,lstm_cnn
0,0,0,0,0,0,0,0,1,1
1,1,1,1,1,1,1,1,1,1
2,1,1,1,1,1,1,1,1,1
3,0,0,0,0,0,0,0,0,0
4,1,1,1,1,1,1,1,1,1


In [63]:
predicted_df['label'] = predicted_df.mode(axis=1)

In [59]:
predicted_df['id'] = df.id.values

In [65]:
predicted_df[['id','label']].to_csv('submit.csv',index=False)